<a href="https://colab.research.google.com/github/mbagci06/lung_covid/blob/main/Ultimate_Learning_CPU_lung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import torch
from PIL import Image
from pydrive.drive import GoogleDrive
from google.colab import drive
from pydrive.auth import GoogleAuth
from google.colab import auth
import torchvision.transforms as transforms
from oauth2client.client import GoogleCredentials
drive.mount('/content/drive')
# BASICS 
import os
import numpy as np
import torch

# Colab 
from pydrive.drive import GoogleDrive
from google.colab import drive
from google.colab import files
from pydrive.auth import GoogleAuth
from google.colab import auth
import torchvision.transforms as transforms
from oauth2client.client import GoogleCredentials
drive.mount('/content/drive')
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# MODEL and TRAIN
import torchvision
import torchvision.models
from torch.utils.data import DataLoader,Dataset
import random
from skimage import io
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import pyplot as plt
from torch import optim
from PIL import Image

device='cuda:0'

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def dice( inputs, targets):
    """dice coefficient calculation function.
        intersection*2/pixelnumber(Inputs)+pixelnumber(targets)

    :param input  : the output of the model.
    :param targets: The reference mask.

    :returns : dice coefficent  .
    :rtype   : float
    """

    # Input can be values rather then 0 and 1 to arrange it
    inputs = torch.where(inputs>=0.5, 1, 0)
    inputs = torch.flatten(inputs)
    targets = torch.flatten(targets)
    # The the part where we have calcualted the function 
    dice = (1e-5+torch.sum(inputs*targets)*2.0) / (torch.sum(targets) + torch.sum(inputs))
    return float(dice)
def iou(y_true, y_pred, smooth = 0.1):
    y_true= torch.where(y_true>=0.5, 1, 0)
    intersection = torch.sum(y_true * y_pred)
    sum_ = torch.sum(y_true) + torch.sum(y_pred)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return jac


def binary_acc(y_true, y_pred, smooth = 0.1):
    y_true= torch.where(y_true>=0.5, 1, 0)
    y_true=torch.flatten(y_true)
    y_pred=torch.flatten(y_pred)
    sum_ = torch.sum(y_true==y_pred)
    acc = ( sum_ ) / (y_pred.shape[0])
    return acc


In [ ]:
%%shell

# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.8.2

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

Cloning into 'vision'...
remote: Enumerating objects: 150039, done.
remote: Counting objects: 100% (2415/2415), done.
remote: Compressing objects: 100% (348/348), done.
remote: Total 150039 (delta 2089), reused 2327 (delta 2058), pack-reused 147624
Receiving objects: 100% (150039/150039), 295.73 MiB | 15.51 MiB/s, done.
Resolving deltas: 100% (132665/132665), done.
Note: checking out 'v0.8.2'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 2f40a483d [v0.8.X] .circleci: Add Python 3.9 to CI (#3063)


In [ ]:

from engine import train_one_epoch, evaluate
import utils
import transforms as T

In [ ]:
!cp /content/drive/MyDrive/Covid/functions.ipynb  /content  

cp: cannot stat '/content/drive/MyDrive/Covid/functions.ipynb': No such file or directory


In [ ]:
%run functions.ipynb

ERROR:root:File `'functions.ipynb.py'` not found.


In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
class kaan_dataset(Dataset): # inherit from torch.utils.data.Dataset
    "Lung and covid-19 sengmentation dataset."

    def __init__(self,root_dir = ("/content/drive/Shared drives/FurkanDataandModels/Furkan_covid/archive"),class_folder='lung',split = "Train", transforms = None , shuffle = True,batch_size=1,
                 sample_size=500 ):
        """
        Args:
        :param root_dir (str): the location of the datasetfolders 
        :param class_folder (str): the lung or covid default to lung 
        :param split (str): the usage purpose of the data
        :param transforms( optional): for normalization  ,default to None
        :param batch_size (int, optional) : default 1 
        :param class (str) : lung or covid : to call the dataset, default to lung:  
        :param sample_size (int) : default to 500
        """

        self.root_dir = root_dir
        self.class_folder=class_folder
        self.split = split # Test Train Val
        self.transforms = transforms
        self.batch_size=batch_size
        self.sample_size=sample_size

        # used to store image and mask file names 
        image_file=[]
        mask_file=[]
        mask_folder=[]
        array_file=[]
        # Label array used to reach folders 
        self.label_array=["/COVID-19","/Non-COVID","/Normal"]
        self.image_folder='/images/'
        # If the dataset called for lung segmentattion  clas folder will be acording to that 
        if class_folder=='lung':
          self.class_folder='/Lung Segmentation Data/Lung Segmentation Data/'
          self.mask_folder='/lung masks/'
        else:
          self.class_folder='/Infection Segmentation Data/Infection Segmentation Data/'
          self.mask_folder='/infection masks/'
        
        # loop iterated 3 times for labels and the data filled with the form of the image ans mask as 
        """array_file example data: /COVID-19/images/covid123.png """
        # Image label/folder/file
        for label_num in range(len(self.label_array)):
          array_file=[]
          path_image=self.root_dir+self.class_folder+self.split+self.label_array[label_num]+self.image_folder
          path_mask=self.root_dir+self.class_folder+self.split+self.label_array[label_num]+self.mask_folder
          # Flatten name_list 
          name_list_image=(os.listdir(path_image))
          name_list_mask=(os.listdir(path_mask))
          data_image=[fName for fName in name_list_image ]
          data_mask=[fName for fName in name_list_mask ]
          # arrange the locations of the images and mask on the right form 
          for x in range(len(data_image)):
            array_file.append('{}{}{}'.format(self.label_array[label_num],self.image_folder,data_image[x]))
          image_file.append(array_file)
          array_file=[]
          for x in range(len(data_mask)):
            array_file.append('{}{}{}'.format(self.label_array[label_num],self.mask_folder,data_mask[x]))
          mask_file.append(array_file)
        # flatten and shorten the size of the dataset 
        self.image_file=[item for sublist in image_file for item in sublist][:sample_size]  
        self.mask_file=[item for sublist in mask_file for item in sublist][:sample_size]  
        self.mask_idx = [ind for ind in range(sample_size) ]
        # Shuffle the dataset 
        if split=='Train' and shuffle==True:
          c = list(zip(self.image_file, self.mask_file,self.mask_idx))
          random.shuffle(c)
          self.image_file, self.mask_file,self.mask_idx = zip(*c)
        

        # Shorten the files after shuffle 
        self.image_file=self.image_file[:sample_size]
        self.mask_file=self.mask_file[:sample_size]
        self.mask_idx=self.mask_idx[:sample_size]
        # Storig the data to the dictionary
        self.data_file = {"Data"  : {"image":self.image_file, "mask": self.mask_file}}
        self.mask_idx ={"idx" : self.mask_idx}
        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    def __len__(self):
        return len(self.mask_idx['idx'])

    def __getitem__(self, idx):
        idx = self.mask_idx['idx'][idx]
        # set index
        # The paths of the images and the masks have called
        img_path=os.path.join(self.root_dir+self.class_folder+self.split+self.data_file['Data']['image'][idx])
        mask_path=os.path.join(self.root_dir+self.class_folder+self.split+self.data_file["Data"]['mask'][idx])

        # Colling the path to reach the images and the masks 
        # The critical part in here is to use 'L' because it helps us to use the image its non int values 
        # For the mask files the images do not  need to be in form of that.  
        img  = Image.open(img_path).convert('L')
        mask = Image.open(mask_path)
        # print(img)
        # print(img_path)
        trans1 = transforms.ToTensor()
        #trans2= transforms.ToTensor()

        img=trans1(img)
        #mask=trans2(mask)
        # The return dictionary 
        mask = np.array(mask)
        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]
        #print(obj_ids)
        # split the color-encoded mask into a set
        # of binary masks
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd
        sample = {'image': img, 'mask': mask}
       
        # Changing the dtype and form of the sample data
        # 
        # sample=trans1(sample)
        if self.transforms:
            sample = self.transforms(img,target)

        
        return img,target

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# The Calling the validaton dataset
val=kaan_dataset(split = "Val",class_folder='covid',shuffle=False,sample_size=200)# The length of the validation dataset to calculate the average value of the loss and accuracy

val_size=val.__len__()
# The Train dataset 
tra=kaan_dataset(split = "Train",class_folder='covid',shuffle=False,sample_size=200)
tra_size=tra.__len__()

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# load a model pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 2  # 1 class (person) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

In [ ]:
batch_size=12
# loading images and masks for mini epochs 
data_loader = torch.utils.data.DataLoader(
        tra, batch_size=batch_size,  num_workers=2, drop_last=True, collate_fn=utils.collate_fn)

data_loader_val = torch.utils.data.DataLoader(
        val, batch_size=batch_size,  num_workers=2, drop_last=True, collate_fn=utils.collate_fn)

In [ ]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
# load a pre-trained model for classification and return
# only the features
backbone = torchvision.models.mobilenet_v2(pretrained=True).features
# FasterRCNN needs to know the number of
# output channels in a backbone. For mobilenet_v2, it's 1280
# so we need to add it here
backbone.out_channels = 1280

# let's make the RPN generate 5 x 3 anchors per spatial
# location, with 5 different sizes and 3 different aspect
# ratios. We have a Tuple[Tuple[int]] because each feature
# map could potentially have different sizes and
# aspect ratios
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))

# let's define what are the feature maps that we will
# use to perform the region of interest cropping, as well as
# the size of the crop after rescaling.
# if your backbone returns a Tensor, featmap_names is expected to
# be [0]. More generally, the backbone should return an
# OrderedDict[Tensor], and in featmap_names you can choose which
# feature maps to use.
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                output_size=7,
                                                sampling_ratio=2)

# put the pieces together inside a FasterRCNN model
model = FasterRCNN(backbone,
                   num_classes=1,
                   rpn_anchor_generator=anchor_generator,
                   box_roi_pool=roi_pooler)



def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model
  

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]

In [ ]:
#device='cpu'
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#device = torch.device('cpu')

model = get_model_instance_segmentation(2).to(device)

for p in model.backbone.parameters():
  p.requires_grad = False

# freeze the fc6 layer in roi_heads
for p in model.roi_heads.box_head.fc6.parameters():
  p.requires_grad = False

# freeze the fc6 layer in roi_heads
for p in model.roi_heads.box_head.fc7.parameters():
  p.requires_grad = False

# move model to the right device

model.to(device)
  # construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size=3,
                                                gamma=0.1)

# let's train it for 10 epochs
num_epochs = 10
trans1 = transforms.ToTensor()
trans2 = transforms.ToTensor()

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth


  0%|          | 0.00/170M [00:00<?, ?B/s]

In [ ]:
for x in range(90):
  train_one_epoch(model, optimizer, data_loader, device, x, 15)
  evaluate(model, data_loader, device)


Epoch: [0]  [ 0/16]  eta: 0:03:18  lr: 0.000034  loss: 3.1528 (3.1528)  loss_classifier: 0.5253 (0.5253)  loss_box_reg: 0.1513 (0.1513)  loss_mask: 2.4453 (2.4453)  loss_objectness: 0.0180 (0.0180)  loss_rpn_box_reg: 0.0129 (0.0129)  time: 12.4357  data: 10.9788  max mem: 3247
Epoch: [0]  [15/16]  eta: 0:00:03  lr: 0.000500  loss: 1.0238 (1.4065)  loss_classifier: 0.2928 (0.3228)  loss_box_reg: 0.0827 (0.0922)  loss_mask: 0.6096 (0.9646)  loss_objectness: 0.0124 (0.0182)  loss_rpn_box_reg: 0.0081 (0.0087)  time: 3.9808  data: 3.2105  max mem: 3274
Epoch: [0] Total time: 0:01:03 (3.9832 s / it)
creating index...
index created!
Test:  [ 0/16]  eta: 0:00:32  model_time: 1.3997 (1.3997)  evaluator_time: 0.3656 (0.3656)  time: 2.0587  data: 0.2910  max mem: 3274
Test:  [15/16]  eta: 0:00:01  model_time: 1.2823 (1.2817)  evaluator_time: 0.3680 (0.3698)  time: 1.7194  data: 0.0620  max mem: 3274
Test: Total time: 0:00:27 (1.7227 s / it)
Averaged stats: model_time: 1.2823 (1.2817)  evaluator_t

In [ ]:
evaluate(model, data_loader, device)

creating index...
index created!
Test:  [ 0/16]  eta: 0:00:18  model_time: 0.7624 (0.7624)  evaluator_time: 0.0838 (0.0838)  time: 1.1414  data: 0.2928  max mem: 4156
Test:  [15/16]  eta: 0:00:00  model_time: 0.7093 (0.7119)  evaluator_time: 0.0665 (0.0682)  time: 0.8469  data: 0.0636  max mem: 4156
Test: Total time: 0:00:13 (0.8496 s / it)
Averaged stats: model_time: 0.7093 (0.7119)  evaluator_time: 0.0665 (0.0682)
Accumulating evaluation results...
DONE (t=0.05s).
Accumulating evaluation results...
DONE (t=0.05s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.358
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.684
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.313
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.008
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.030
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | max

In [ ]:
# The number of the epoch 
epoch_number=90
# list for the calculating validation Loss
val_loss_array=[]
# list for the calculating train Loss
train_loss_array=[]
# list for the calculating validation accuracy
acc_val_array=[]
# list for the calculating train accuracy
acc_train_array=[]
for t in range(epoch_number):
    # reset the loss and the accuracy values each epoch 
    val_loss=0
    train_loss=0
    acc_val=0
    acc_train=0
    cnt=0
    for idx, target in enumerate(data_loader):
        img=target[0]
        # Train data preparing 
        cnt=cnt+1
        #print(img)
          #imgs , true_masks = sample['image'],sample['mask']
        #img = img.to(device=device)
          #imgs=torch.reshape(img, (batch_size,3,256,256))
          #true_masks=Negative_mask(true_masks,imgs.size(dim=0))
        model.train()
          # zero the parameter gradients
        optimizer.zero_grad()
          #true_masks=true_masks.to('cpu').numpy()
          #targ=target(true_masks,cnt)
        img = list(image.to(device) for image in img)
        #img = img.to(device=device)
        targets = [{k: v.to(device)  for k, v in t.items()} for t in target[1]]
        #targets = [{k: torch.squeeze(v).to(device) for k, v in target[1].items()} ]
        #targets=target[1]
        masks_pred = model(img,targets)
          #true_masks = true_masks.to(device=device)
          
  

      #  """forward"""
    #true_masks=true_masks.to('cpu').numpy()
    #targ=target(true_masks,x)
    #targets = [{k: v.to(device) for k, v in targ.items()} ]
    #masks_pred = model(imgs,targets)
        

        with torch.set_grad_enabled(True):
            # Train data evaluation
              #loss = criterion(masks_pred, true_masks)
              loss=masks_pred["loss_mask"]
              train_loss += loss.item()
            # Updating the params
              loss.backward()
              optimizer.step()
            # Acc calc train
              #acc_train+=dice( masks_pred, true_masks)
    for idx, target in enumerate(data_loader):
        # Validation data preparing  and calling 
        # calcel gradient feature 
        model.eval()
        img=target[0]
        # Call iamge and mask

        img = list(image.to(device)  for image in img)
        #img = img.to(device=device)
        targets = [{k: v.to(device)  for k, v in t.items()} for t in target[1]]
        # imgs , true_masks = sample['image'],sample['mask']

        # imgs = imgs.to(device=device)
        # true_masks=Negative_mask(true_masks,imgs.size(dim=0))
        # true_masks = true_masks.to(device=device)

        # zero the parameter gradients
        optimizer.zero_grad()
        masks_pred_val = model(img,targets)
        with torch.set_grad_enabled(False):
            # Validation evaluation 
            #train_loss += loss.item()
            for j in range(batch_size):
              a=torch.zeros(256,256).to(device) 
              for i in range(100):
                a=a+masks_pred_val[j]['masks'][i][0,:,:]
            acc_val += dice(a,targets[j]['masks'][0,:,:])
            # masks_pred = model(imgs)
            # masks_pred = masks_pred.to(device=device)
            # loss_val = criterion(masks_pred, true_masks)
            # acc_val +=dice( masks_pred, true_masks)
            # val_loss += loss_val.item()

    #batch_size
    print('Epoch :',t+1,'/',str(epoch_number))
    print('Validation Loss:v',(val_loss)/(val_size/batch_size))
    print('Validation Acc:',(acc_val)/(val_size/batch_size))
    print('Train Acc:',(acc_train)/(tra_size/batch_size))
    print('Train  Loss   :t',(train_loss)/(tra_size/batch_size))
    # Store epoch progress
    val_loss_array.append((val_loss)/(val_size/batch_size))
    train_loss_array.append((train_loss)/(tra_size/batch_size))
    acc_val_array.append((acc_val)/(val_size/batch_size))
    acc_train_array.append((acc_train)/(tra_size/batch_size))

IndexError: ignored

In [ ]:
masks_pred

In [ ]:
masks_pred_val

In [ ]:
a=torch.zeros(256,256)
for i in range(100):
  a=a+masks_pred_val[0]['masks'][i][0,:,:].detach().numpy()

In [ ]:
masks_pred_val[]['masks'][i][0,:,:].detach().numpy()